# CAMEL Model for all the cooperatives

## Functions

In [1]:
import pandas as pd

### Capital Adequacy

In [2]:
# Loss of Assets
def loss_of_assets(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 300000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]
    
    if account_coop1.empty or account_coop2.empty:
        return 0
    
    value1 = account_coop1.item()
    value2 = account_coop2.item()
    print(value1, value2)
    
    return value1 / value2 if value2 != 0 else 0

# Solvency Ratio
def solvency_ratio():
    return 1

# Indicator of the relationship between the non-reducible minimum social contributions and the Share Capital
def min_social_contrib_social_capital(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 311000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]

    if account_coop1.empty or account_coop2.empty:
        return 0
    
    value1 = account_coop1.item()
    value2 = account_coop2.item()
    
    return value1 / value2 if value2 != 0 else 0


# --- Sum the values of every row 'i' in a specific company
def summation(excel_file, index_list, company_name, year, month):
    suma = 0
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    for i in index_list:
        mask = (excel_file['CUENTA'] == i) & (excel_file['fecha'] == date_)
        account_coop = excel_file.loc[mask, company_name]
        if account_coop.empty or account_coop.isna().all():
            value = 0
        else:
            value = account_coop.iloc[0]  # o .sum() si esperas múltiples filas
        suma += value
    return suma


# Indicator of the relationship between Institutional Capital and Total Assets
def capital_contribution_ratio(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    capital_inst_indices = [320000, 330000, 340000]
    capital_inst = summation(excel_file, capital_inst_indices, company_name, year, month)
    
    mask_total_assets = (excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == date_)
    total_assets = excel_file.loc[mask_total_assets, company_name]
    if total_assets.empty:
        return 0
    total_assets_ = list(total_assets)
    value1 = capital_inst
    value2 = total_assets_[0]

    return value1 / value2 if value2 != 0 else 0

### Assets Quality

In [3]:
# Functions for quality indicator by risk with penalties
def gross_portfolio(excel_file, company_name, year, month):
    gross_portfolio_indices = [
        140400, 140500, 141100, 141200, 144100, 144200,
        144800, 145400, 145500, 146100, 146200, 146900
    ]
    return summation(excel_file, gross_portfolio_indices, company_name, year, month)

def risk_quality_indicator(excel_file, company_name, year, month):
    risk_portfolio_indices = [
        140410, 140415, 140420, 140425, 140510, 140515, 140520, 140525,
        141110, 141115, 141120, 141125, 141210, 141215, 141220, 141225,
        144110, 144115, 144120, 144125, 144210, 144215, 144220, 144225,
        144810, 144815, 144820, 144825, 145410, 145415, 145420, 145425,
        145510, 145515, 145520, 145525, 146110, 146115, 146120, 146125,
        146210, 146215, 146220, 146225, 146910, 146915, 146920, 146925,
        146935, 146940, 146945, 146950
    ]
    qualified_portfolio = summation(excel_file, risk_portfolio_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file,company_name, year, month)

    return qualified_portfolio / gross_portf if gross_portf != 0 else 0

def risk_quality_with_writeoffs(excel_file, company_name, year, month):
    # total qualified portfolio
    qualified_portfolio = risk_quality_indicator(excel_file, company_name, year, month)
    # writeoffs
    writeoffs = excel_file.loc[(excel_file['CUENTA'] == 831015) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    total_with_writeoffs_indices = [
        140400, 140500, 141100, 141200, 144100, 144200, 144800,
        145400, 145500, 146100, 146200, 146900, 831015
    ]
    total_with_writeoffs = summation(excel_file, total_with_writeoffs_indices, company_name, year, month)
    return (qualified_portfolio + writeoffs) / total_with_writeoffs if total_with_writeoffs != 0 else 0

# Total Portfolio at Risk Coverage Indicator
def total_risk_coverage_indicator(excel_file, company_name, year, month):
    deterioration_indices = [140800, 144500, 145100, 145800, 146500, 146800, 147100]
    provisions = summation(excel_file, deterioration_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file, company_name, year, month)
    return provisions / gross_portf if gross_portf != 0 else 0

# Productive asset
def productive_assets_ratio(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    total_assets = excel_file.loc[(excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    return productive_assets / total_assets if total_assets != 0 else 0

# Individual Coverage Indicator of the Unproductive Portfolio for the At-Risk Portfolio
def individual_coverage_nonproductive_portfolio(excel_file, company_name, year, month):
    provisions_cde_indices = [
        140815, 140820, 140825,
        144525, 144530, 144535, 144540, 144545, 144550,
        145115, 145120, 145125,
        145825, 145830, 145835, 145840, 145845, 145850,
        146525, 146530, 146535, 146540, 146545, 146550,
        147115, 147120, 147125, 147140, 147145, 147150
    ]
    provisions = summation(excel_file, provisions_cde_indices, company_name, year, month)

    overdue_portfolio_indices = [
        140415, 140420, 140425, 140515, 140520, 140525,
        141115, 141120, 141125, 141215, 141220, 141225,
        144115, 144120, 144125, 144215, 144220, 144225,
        144815, 144820, 144825, 145415, 145420, 145425,
        145515, 145520, 145525, 146115, 146120, 146125,
        146215, 146220, 146225, 146915, 146920, 146925,
        146940, 146945, 146950
    ]
    overdue = summation(excel_file, overdue_portfolio_indices, company_name, year, month)

    return provisions / overdue if overdue != 0 else 0

### Managerial Quality

In [4]:
# Operating Financial Margin Indicator
def financial_margin_operation(excel_file, company_name, year, month):
    positive_margin = excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    negative_margin1 = excel_file.loc[(excel_file['CUENTA'] == 610000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    negative_margin2 = excel_file.loc[(excel_file['CUENTA'] == 700000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    negative_margin = negative_margin1 + negative_margin2
    sales_income = excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    return (positive_margin - negative_margin) / sales_income if sales_income != 0 else 0

# Operating Margin Indicator
def operational_margin(excel_file, company_name, year, month):
    income_pos_indices = [410000, 422500]
    income_neg_indices = [610000, 700000, 510500, 510700, 511000, 511500, 540000]
    sales_indices = [410000, 422500]
    income_pos = summation(excel_file, income_pos_indices, company_name, year, month)
    income_neg = summation(excel_file, income_neg_indices, company_name, year, month)
    sales_income = summation(excel_file, sales_indices, company_name, year, month)
    return (income_pos - income_neg) / sales_income if sales_income != 0 else 0

# Indicator of the relationship between financial obligations and total liabilities
def financial_obligations_ratio(excel_file, company_name, year, month):
    obligations = excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    total_liabilities = excel_file.loc[(excel_file['CUENTA'] == 200000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    return obligations / total_liabilities if total_liabilities != 0 else 0

# Balance structure
def balance_structure(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    interest_liabilities_indices = [210000, 230000]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    interest_liabilities = summation(excel_file, interest_liabilities_indices, company_name, year, month)
    return productive_assets / interest_liabilities if interest_liabilities != 0 else 0

### Earnings Strength

In [5]:
# Return on equity indicator
def equity_indicator():
    return 1

# Net margin indicator
def net_margin_indicator(excel_file, company_name, year, month):
    net_surplus = excel_file.loc[(excel_file['CUENTA'] == 530000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    incomes_indices = [410000, 422500]
    total_incomes = summation(excel_file, incomes_indices, company_name, year, month)
    return net_surplus/total_incomes if total_incomes != 0 else 0

# Indicator of return on invested capital
def return_on_invested_capital(excel_file, company_name, year, month):
    value_january = excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name]
    months = [y for y in range(1, 13, 1)]
    values_every_month = 0
    for m in months:
        month = excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == f'{int(year-1)}-{int(m)}-01'),company_name]
        values_every_month += month

    media_last_year = values_every_month/12

    return (media_last_year + value_january)/2

### Liquidicy Efficiency

In [6]:
# Liquidity Risk Indicator
def liquidity_risk():
    return 1

## Create the csv for all the cooperatives

In [7]:
coops = pd.read_excel("resultados/Datos_2013_2025_cooperativas.xlsx")
coops

,CUENTA,NOMBRE CUENTA,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA,CAJA COOPERATIVA CREDICOOP,COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA,...,COOPERATIVA DE AHORRO Y CREDITO COOMPARTIR,COOPERATIVA DE AHORRO Y CREDITO BERLIN,COOPERATIVA DE AHORRO Y CREDITO DE AIPE,COOPERATIVA DE AHORRO Y CREDITO DE SANTANDER LIMITADA,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA DE AHORRO Y CREDITO FINANCIAFONDOS,COOPERATIVA DE AHORRO Y CREDITO DE DROGUISTAS DETALLISTAS,COOPERATIVA DE AHORRO Y CREDITO COLANTA,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO AFROAMERICANA,fecha
0,100000,ACTIVO,3.387150e+11,1.560676e+11,9.879448e+10,1.064674e+10,7.021719e+10,1.417133e+11,8.491366e+10,2.198054e+10,...,4.312265e+09,4.443368e+10,2.850488e+10,1.641308e+12,3.832060e+11,1.559309e+10,2.597647e+11,3.174706e+11,7.498950e+09,2022-12-01
1,110000,EFECTIVO Y EQUIVALENTE AL EFECTIVO,1.566943e+10,2.669495e+10,1.918907e+09,1.552585e+09,4.685907e+09,1.434015e+10,6.147909e+09,2.294268e+09,...,1.192062e+08,2.639965e+09,2.223880e+09,2.443691e+11,7.403156e+09,2.817348e+08,3.586048e+10,2.764945e+10,5.507006e+08,2022-12-01
2,110500,CAJA,8.229352e+08,1.100000e+06,3.589941e+08,1.718631e+08,1.241549e+08,6.995092e+08,9.522290e+07,1.118834e+07,...,2.765270e+07,2.401334e+08,2.473210e+08,1.236956e+10,2.169476e+09,0.000000e+00,2.000000e+06,5.735745e+09,4.744467e+08,2022-12-01
3,110505,CAJA GENERAL,8.180852e+08,0.000000e+00,3.589941e+08,1.718631e+08,1.146549e+08,6.995092e+08,9.522290e+07,1.048834e+07,...,2.732270e+07,2.394334e+08,2.473210e+08,1.235737e+10,2.168001e+09,0.000000e+00,0.000000e+00,5.732195e+09,4.738467e+08,2022-12-01
4,110510,CAJA MENOR,4.850000e+06,1.100000e+06,0.000000e+00,0.000000e+00,9.500000e+06,0.000000e+00,0.000000e+00,7.000000e+05,...,3.300000e+05,7.000000e+05,0.000000e+00,1.218300e+07,1.475000e+06,0.000000e+00,2.000000e+06,3.550000e+06,6.000000e+05,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292698,935000,OTRAS ACREEDORAS DE CONTROL,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.934962e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2014-12-01
292699,960000,ACREEDORAS CONTINGENTES POR CONTRA,2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292700,960500,ACREEDORAS CONTINGENTES POR CONTRA (DB),2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292701,980000,ACREEDORAS DE CONTROL POR CONTRA,2.619920e+10,3.159206e+11,7.700000e+09,1.299760e+09,6.160000e+09,0.000000e+00,1.232000e+10,3.080000e+09,...,1.274480e+09,3.654957e+09,1.386000e+09,8.014403e+10,8.129557e+09,1.289659e+09,5.000000e+09,2.140509e+10,2.266800e+09,2014-12-01


In [8]:
coops.fillna(0, inplace=True)

In [9]:
date_ = pd.Timestamp(f'{int(2019)}-{int(11)}-01')
value = list(coops.loc[(coops['CUENTA'] == 310000) & (coops['fecha'] == date_),'COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS'])
print(value[0])

107745281591.27


In [10]:
keys = list(coops.columns)
keys.remove("CUENTA")
keys.remove('NOMBRE CUENTA')
print(keys)

['COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS', 'COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL', 'COOPERATIVA PARA EL BIENESTAR SOCIAL', 'COOPERATIVA FINANCIERA SAN FRANCISCO', 'COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA', 'COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA', 'CAJA COOPERATIVA CREDICOOP', 'COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA', 'FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS', 'COOPERATIVA DEL MAGISTERIO', 'COOPERATIVA DE AHORRO Y CREDITO DE CHIPAQUE', 'COOPERATIVA DE AHORRO Y CREDITO UNIVERSIDAD SANTO TOMAS', 'CAJA COOPERATIVA PETROLERA', 'COOPERATIVA TEXAS LTDA', 'COOPERATIVA DE LOS TRABAJADORES DEL INSTITUTO DE LOS SEGUROS SOCIALES', 'COOPERATIVA DE TRABAJADORES DE BAVARIA DIRECCION Y VENTAS LTDA', 'COPROCENVA COOPERATIVA DE AHORRO Y CREDITO', 'COOPERATIVA DE AHORRO Y CREDITO DE TENJO', 'COOPERATIVA DE TRABAJADORES Y PENSIONADOS DE LA E.A.A.B.', 'COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA', 'COOPERATIVA MULTIA

In [11]:
first_columns = ["CAMEL", "SUB_ITEM"]
camel_table_columns = first_columns + keys
print(camel_table_columns)

['CAMEL', 'SUB_ITEM', 'COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS', 'COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL', 'COOPERATIVA PARA EL BIENESTAR SOCIAL', 'COOPERATIVA FINANCIERA SAN FRANCISCO', 'COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA', 'COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA', 'CAJA COOPERATIVA CREDICOOP', 'COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA', 'FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS', 'COOPERATIVA DEL MAGISTERIO', 'COOPERATIVA DE AHORRO Y CREDITO DE CHIPAQUE', 'COOPERATIVA DE AHORRO Y CREDITO UNIVERSIDAD SANTO TOMAS', 'CAJA COOPERATIVA PETROLERA', 'COOPERATIVA TEXAS LTDA', 'COOPERATIVA DE LOS TRABAJADORES DEL INSTITUTO DE LOS SEGUROS SOCIALES', 'COOPERATIVA DE TRABAJADORES DE BAVARIA DIRECCION Y VENTAS LTDA', 'COPROCENVA COOPERATIVA DE AHORRO Y CREDITO', 'COOPERATIVA DE AHORRO Y CREDITO DE TENJO', 'COOPERATIVA DE TRABAJADORES Y PENSIONADOS DE LA E.A.A.B.', 'COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA'

In [12]:
mask = coops == '-'
count = mask.sum().sum()

print(f"Cantidad de celdas con '-': {count}")


Cantidad de celdas con '-': 0


In [13]:
coops = coops.replace('-', 0)

for col in coops.columns:
    if col not in ['CUENTA', 'fecha']:
        coops[col] = pd.to_numeric(coops[col], errors='coerce').fillna(0)


In [14]:
values = []
# add the values for C, for each year
years = [y for y in range(2014, 2025, 1)]
months = [m for m in range(1,13,1)]
# cooperatives
for company in keys:
    row = []
    # calculate the value for every year, for the cooperative 'c'
    for y in years:
        for m in months:
            value1 = loss_of_assets(coops, company, y, m)
            value2 = solvency_ratio()
            value3 = min_social_contrib_social_capital(coops, company, y, m)
            value4 = capital_contribution_ratio(coops, company, y, m)
            row.append(value1)
            row.append(value2)
            row.append(value3)
            row.append(value4)
    values.append(row)

print(values)

85936714009.52 66146765786.02
86594642626.34 66657680205.63
86302085561.72 68409929227.2


ValueError: can only convert an array of size 1 to a Python scalar